### Collectinng Data
Gets player statistics & gameweek performance from the FPL API & saves it to the database. This class does one thing only. SRP

### Solution
- Connects to the FPL API 
- gets current season player information
- gets gameweeks performance stats
- Saves data to the SQLite database

### API
- `/api/bootstrap-static/`, gets all players, teams & current gameweek info
- `/api/event/{gameweek}/live/`, gets performance stats for a specific gameweek

### References 
https://fantasy.premierleague.com/api/

https://docs.python-requests.org/

In [2]:
# Collecting data from the FPL API
# importing requests for making http requests to the API
# importing sqlite3 for saving data to the database
# importing time for adding delays between the API requests
import requests
import sqlite3
import time

class DataCollector:
    """
    Collects data from the FPL API

    gets the player information & gameweeke performance data 
    & saves it to the SQLite database 
    """

    def __init__(self):
        """Creating database path & link to the API URL """
        self.db = 'fpl_data.db'
        self.api = "https://fantasy.premierleague.com/api"

    def Getplayers(self):
        """Getting the players & saving it to the database"""
        print("Getting players")

        # Getting data from FPL API
        data = requests.get(f"{self.api}/bootstrap-static/").json()

        # Connecting to the database
        conn = sqlite3.connect(self.db)
        cursor = conn.cursor()

        # 1=GK, 2=DEF, 3=MID, 4=FWD
        positions = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}
        # moving team IDs to team names
        teams = {t['id']: t['name'] for t in data['teams']}

        # Saving each player to the database
        for p in data['elements']:
            cursor.execute(
                'INSERT OR REPLACE INTO players VALUES (?, ?, ?, ?, ?)',
                (p['id'], p['web_name'], teams[p['team']], 
                 positions[p['element_type']], p['now_cost'] / 10)
            )
        
        # Save and close
        conn.commit()
        conn.close()
        # printing results
        print(f"Saved {len(data['elements'])} players")